In [25]:
import duckdb

MinIO Setup

In [26]:
minioAuth = {
    'accessKey': 'minioadmin',
    'secretKey': 'miniopassword',
    'bucket': 'data-dapodik-kemendikdasmen',
    'endpoint': 'http://localhost:9000'
}

Path yang Digunakan

In [27]:
localCatalogFile = "../metadata/dapodikMetadata.ducklake"
minioNewDataFolder = f"s3://{minioAuth['bucket']}/ducklake/JawaBarat/" # Folder tempat DuckLake menyimpan data barunya
currentMiniofilePath = f"s3://{minioAuth['bucket']}/JawaBarat/dataDKIJakarta.parquet" # LOKASI FILE PARQUET LIVE ANDA
print(currentMiniofilePath)

s3://data-dapodik-kemendikdasmen/JawaBarat/dataDKIJakarta.parquet


DuckDB Connection & Install Extension 

In [28]:
con = duckdb.connect()
con.sql("INSTALL httpfs; LOAD httpfs;")
con.sql("INSTALL ducklake; LOAD ducklake;")

Set MinIO Credentials

In [29]:
con.sql(f"SET s3_endpoint = '{minioAuth['endpoint'].replace('http://', '')}';")
con.sql("SET s3_use_ssl = false;") 
con.sql(f"SET s3_access_key_id = '{minioAuth['accessKey']}';")
con.sql(f"SET s3_secret_access_key = '{minioAuth['secretKey']}';")
con.sql("SET s3_url_style = 'path';")

ATTACH ke DuckLake

In [30]:
con.sql(f"ATTACH 'ducklake:{localCatalogFile}' AS jakartaLake (DATA_PATH '{currentMiniofilePath}');")
con.sql("USE jakartaLake;")

### Langkah 2: Memuat Data Live ke DuckLake (Metadata Management)
Kita akan menggunakan perintah CREATE TABLE AS SELECT untuk membaca file Parquet live Anda dan menuliskannya ke Tabel DuckLake baru.

Perintah ini akan melakukan dua hal:
1. Membuat Schema
1. Membaca konten dari MINIO_LIVE_PARQUET (Parquet yang sudah ada).
2. Menulis data tersebut dalam format yang dikelola DuckLake (dengan commit log metadata) ke dalam Data Path DuckLake Anda (MINIO_DATA_PATH).

In [31]:
con.sql("CREATE SCHEMA IF NOT EXISTS staging;")

2. Load Data

In [32]:
loadLiveDataQuery = f"""
    -- DuckLake akan menghapus metadata tabel lama, menghapus file fisik di MinIO,
    -- kemudian membuat tabel baru dan menulis data dari read_parquet.
    CREATE OR REPLACE TABLE staging.dataDKIJakarta AS 
    SELECT 
        * FROM 
        read_parquet('{currentMiniofilePath}');
"""

con.sql(loadLiveDataQuery)

3. Verifikasi Snapshot Awal

In [33]:
print("\n--- Riwayat Snapshot Awal ---")
con.sql("SELECT * FROM ducklake_snapshots('jakartaLake');").df()


--- Riwayat Snapshot Awal ---


,snapshot_id,snapshot_time,schema_version,changes,author,commit_message,commit_extra_info
0,0,2025-12-02 21:50:52.889002+07:00,0,{'schemas_created': ['main']},None,None,None
1,1,2025-12-02 21:50:53.047768+07:00,1,{'schemas_created': ['staging']},None,None,None
2,2,2025-12-02 21:50:53.085552+07:00,2,"{'tables_created': ['staging.dataDKIJakarta'],...",None,None,None
3,3,2025-12-02 22:03:11.206043+07:00,3,"{'tables_created': ['staging.dataDKIJakarta'],...",None,None,None
4,4,2025-12-02 22:08:53.181807+07:00,4,"{'tables_created': ['staging.dataDKIJakarta'],...",None,None,None
5,5,2025-12-02 22:09:02.453704+07:00,5,"{'tables_created': ['staging.dataDKIJakarta'],...",None,None,None


In [34]:
con.close()